In [1]:
%reload_ext autoreload
%autoreload 2
import os
import yaml
if "NeuroVoice" != os.path.basename(os.getcwd()):
    os.chdir("../")
from pipelines.train import TrainPipeline
from IPython.display import Audio, display
from models.tts.xtts_finetuner import XTTSFinetuner, OUTPUT_SAMPLE_RATE

config_path = "configs/train/shtefanov.yaml"
if not os.path.exists(config_path):
    print("Файл не существует! Исполни команду 'python utils/create_config.py' для создание нового конфига.")
config = yaml.safe_load(open(config_path))
trainpipe = TrainPipeline(config)
trainpipe._preprocess_data()
# trainpipe.train()

/home/egatiyatullin/repos/NeuroVoice/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/egatiyatullin/repos/NeuroVoice/.venv/lib/python3.10/site-packages/librosa/core/intervals.py:8: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename
/home/egatiyatullin/repos/NeuroVoice/.venv/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/egatiyatullin/repos/NeuroVoice/.venv/lib/pyth

😞 Mandatory files are not ready.
😞 wavs/ folder is empty.
⚙️ Starting process of creating files... (REQUIRES GPU!)


/home/egatiyatullin/repos/NeuroVoice/transcriber/asr_transcriber.py:149: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with amp_ctx(enabled=use_amp):


VAD segments num before merging: 223 and after: 213


  8%|▊         | 16/213 [00:39<08:02,  2.45s/it]


KeyboardInterrupt: 

In [ ]:


        merged = []
        current_group = []
        target_duration = ((min_duration + 10.0) / 2)*sr  # Target middle of range
        max_gap = max_gap*sr
        for i, segment in enumerate(segments):
            current_duration = sum(s["end"] - s["start"] 
                                for s in current_group) if current_group else 0

            # If this is a continuation of current group
            # print(f"i:{i}, (segment['start'] - current_group[-1]['end']) {(segment['start'] - current_group[-1]['end']) if current_group else None} ,  max_gap:{max_gap}")
            if current_group and (segment["start"] - current_group[-1]["end"]) <= max_gap:
                # Check if adding this segment gets us closer to target duration
                new_duration = current_duration + (segment["end"] - segment["start"])
                # print(f"i:{i}, new_duration:{new_duration}, abs(new_duration - target_duration) {abs(new_duration - target_duration)/sr} < abs(current_duration - target_duration) {abs(current_duration - target_duration)/sr}?")
                if abs(new_duration - target_duration) < abs(current_duration - target_duration):
                    current_group.append(segment)
                else:
                    # Save current group and start new one
                    merged_segment = {
                        "start": current_group[0]["start"],
                        "end": current_group[-1]["end"]
                    }
                    merged.append(merged_segment)
                    current_group = [segment]
            else:
                # Save previous group if it exists
                if current_group:
                    merged_segment = {
                        "start": current_group[0]["start"],
                        "end": current_group[-1]["end"]
                    }
                    merged.append(merged_segment)
                current_group = [segment]
                # if i==2:
                #     break

        # Handle last group
        if current_group:
            merged_segment = {
                "start": current_group[0]["start"],
                "end": current_group[-1]["end"]
            }
            merged.append(merged_segment)
        print(f"VAD segments num before merging: {len(segments)} and after: {len(merged)}")
    

In [6]:
import torchaudio
from IPython.display import Audio, display
wav, sr = torchaudio.load("data/divertito/audio/voice1.wav")
display(Audio(wav, rate=sr))

# 3. Проверяем обученную модель

In [5]:
checkpoint_path="out/divertito/xtts/best_model_v2.pth"
base_model_path = trainpipe.finetuner.load_base_model()
model = trainpipe.finetuner.load_model(
    trainpipe.device,
    os.path.join(base_model_path,"config.json"),
    checkpoint_path,
    os.path.join(base_model_path, "vocab.json"),
   os.path.join(base_model_path, "speakers_xtts.pth")
)

Fetching 18 files:   0%|          | 0/18 [00:00<?, ?it/s]/home/egatiyatullin/repos/NeuroVoice/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:982: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(
Fetching 18 files: 100%|██████████| 18/18 [00:00<00:00, 402.95it/s]


In [10]:
# Generate audio
audio = trainpipe.finetuner.generate_audio(
    model,
    "Это, конечно, потешно блядь. Ты сладкий бубочка, а не уважающий себя мужчина, хахах",
    config["data"]["reference_audio"]
)
display(Audio(audio,  rate = OUTPUT_SAMPLE_RATE))

NameError: name 'OUTPUT_SAMPLE_RATE' is not defined